In [19]:
## Set up Conda environment and attach copilot-playground kernel to this notebook

# pip install --user ipykernel
# conda create -n copilot-playground
# conda activate copilot-playground

# conda install ipykernel
# conda install postgresql
# conda install psycopg2
# conda install matplotlib
# conda install plotly
# conda install scipy
# conda install sklearn
# conda install langchain
# conda install openai
# conda install python-dotenv

# conda install postgresql
# conda install psycopg2

In [45]:
## Set up .env file in the same directory as this notebook, with the following 3 variables

import os
from dotenv import load_dotenv

load_dotenv(".env")

open_ai_api_key = os.getenv("OPEN_AI_API_KEY")
normandy_db_username = os.getenv("DB_USER_NAME")
normandy_db_local_port = os.getenv("DB_PORT")

print(normandy_db_username)
print(normandy_db_local_port)

aditya.gunda@getathelas.com
54329


In [56]:
import psycopg2

# NOTE: Make sure that you have the Cloud SQL Auth Proxy running locally for Normandy DB at the provided port.

def connect_to_db():
    try:
        connection = psycopg2.connect(
            user=normandy_db_username,
            password="",
            host="localhost",
            port=normandy_db_local_port,
            database="normandy"
        )
        return connection
    except Exception as error:
        print(f"Error while connecting to PostgreSQL: {error}")

def fetch_data(query):
    connection = connect_to_db()
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        records = cursor.fetchall()
        return records
    except Exception as error:
        print(f"Error fetching data: {error}")
    finally:
        cursor.close()
        connection.close()


In [41]:
import json

def lookup_claim_denial_info(claim_id: int):
  def map_data_to_dicts(data):
    keys = [
        "claim_submission_id", "cpt_code", "modifiers", "place_of_service_code",
        "revenue_code", "insurance_payout", "smartpay_pr", 
        "claim_adjustment_reason_codes", "remittance_advice_remark_codes"
    ]
    return [dict(zip(keys, row)) for row in data]

  data = fetch_data("""
          WITH payout AS (
            SELECT
                csp.claim_submission_id,
                csp.procedure_id,
                csp.modifiers,
                v.remit_insurance_priority::text,
                SUM(COALESCE(v.provider_paid_amount_in_usd_cents, 0))::float / 100 as insurance_payout,
                SUM(COALESCE(v.total_adjustments_smartpay_pr_in_usd_cents, 0))::float / 100 as smartpay_pr
            FROM
                claim_submissions_procedures csp
            JOIN
                all_procedure_details_view v ON (
                            v.claim_submission_id = csp.claim_submission_id
                        AND v.procedure_id = csp.procedure_id
                    )
            WHERE
                csp.claim_submission_id = '{claim_id}'
          GROUP BY 1,2,3,4
        )
        , carc_rarc AS (
          SELECT
              pr.procedure_id,
              ARRAY_AGG(CONCAT(carc.code,' - ', carc.description)) as claim_adjustment_reason_codes,
              ARRAY_AGG(CONCAT(rarc.code, ' - ', rarc.description)) as remittance_advice_remark_codes
          FROM
                  claim_submissions cs
          LEFT JOIN
                  claim_reconciliations crecon ON cs.id = crecon.claim_submission_id
          LEFT JOIN
                  claim_remittances rem ON rem.claim_reconciliation_id = crecon.id
          LEFT JOIN
                  procedure_remittances pr ON pr.claim_remittance_id = rem.id
          LEFT JOIN
                  remittance_advice_remark_codes_procedure_remittances rarcpr ON rarcpr.procedure_remittances_id = pr.id
          LEFT JOIN
                  remittance_advice_remark_codes rarc ON rarc.code = rarcpr.remittance_advice_remark_code
          LEFT JOIN
                  procedure_adjustments pa ON pa.procedure_remittance_id = pr.id
          LEFT JOIN
                  claim_adjustment_reason_codes carc ON carc.code = pa.claim_adjustment_reason_code
          WHERE
              cs.id = '{claim_id}'
              AND carc.code NOT IN ('45')
          GROUP BY
              1
        )

        SELECT
            payout.claim_submission_id,
            cc.code as cpt_code,
            payout.modifiers,
            proc.place_of_service_code,
            rc.code as revenue_code,
            payout.insurance_payout,
            payout.smartpay_pr,
            cr.claim_adjustment_reason_codes,
            cr.remittance_advice_remark_codes
        FROM
            payout
        JOIN
            procedures proc ON proc.id = payout.procedure_id
        JOIN
            cpt_codes cc ON cc.id = proc.cpt_code_id
        LEFT JOIN
            revenue_codes rc ON proc.revenue_code_id = rc.id
        LEFT JOIN
            carc_rarc cr ON cr.procedure_id = proc.id;
    """.format(claim_id=claim_id)
  )

  mapped_data = map_data_to_dicts(data)
  return json.dumps(mapped_data, indent=4)


In [70]:
import json 

def lookup_approved_claims_with_cpt_code(cpt_code: str):
  def map_data_to_dicts(data):
    keys = [
        "claim_submission_id", "cpt_codes", "place_of_service_code",
        "revenue_code", "insurance_payout", "smartpay_pr"
    ]
    
    return [dict(zip(keys, row)) for row in data]

  data = fetch_data("""
    WITH relevant_claims AS (
      SELECT
          ARRAY_AGG(DISTINCT claim_submission_id) as claim_submission_ids
      FROM
          claim_submissions_procedures csp
      JOIN
        procedure_payout_view ppv ON ppv.procedure_id = csp.procedure_id
      JOIN
        cpt_codes cc ON csp.cpt_code_id = cc.id
      WHERE
          cc.code = '{cpt_code}'
          AND COALESCE(ppv.provider_paid_amount_in_usd_cents, 0) > 0
    ),
    payout AS (
      SELECT
          csp.claim_submission_id,
          csp.procedure_id,
          csp.modifiers,
          v.remit_insurance_priority::text,
          SUM(COALESCE(v.provider_paid_amount_in_usd_cents, 0))::float / 100 as insurance_payout,
          SUM(COALESCE(v.total_adjustments_smartpay_pr_in_usd_cents, 0))::float / 100 as smartpay_pr
      FROM
          claim_submissions_procedures csp
      JOIN
          all_procedure_details_view v ON (
                  v.claim_submission_id = csp.claim_submission_id
                  AND v.procedure_id = csp.procedure_id
              )
      WHERE
        csp.claim_submission_id IN (SELECT unnest(claim_submission_ids) FROM relevant_claims)
      GROUP BY 1,2,3,4
      HAVING SUM(COALESCE(v.provider_paid_amount_in_usd_cents, 0))::float > 0 OR SUM(COALESCE(v.total_adjustments_smartpay_pr_in_usd_cents, 0))::float > 0
      ORDER BY 1 DESC
    ) 

    SELECT
        payout.claim_submission_id,
        ARRAY_AGG(CONCAT(cc.code, '-', payout.modifiers)) as cpt_codes,
        ARRAY_AGG(DISTINCT proc.place_of_service_code) as place_of_service_code,
        ARRAY_AGG(DISTINCT rc.code) as revenue_code,
        SUM(COALESCE(payout.insurance_payout, 0)) as insurance_payout,
        SUM(COALESCE(payout.smartpay_pr, 0)) as smartpay_pr
    FROM
        payout
    JOIN
        procedures proc ON proc.id = payout.procedure_id
    JOIN
        cpt_codes cc ON cc.id = proc.cpt_code_id
    LEFT JOIN
        revenue_codes rc ON proc.revenue_code_id = rc.id
    GROUP BY
      1
    LIMIT 50;
  """.format(cpt_code=cpt_code)
  )

  mapped_data = map_data_to_dicts(data)
  return json.dumps(mapped_data, indent=4)

In [87]:
from langchain import OpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools = [
    Tool(
        name="Lookup Claim Procedure Denial Info",
        func=lookup_claim_denial_info,
        description="""
          Useful for when you need get a claim's cpt codes and their denial reasons.
          Claims may have multiple procedures.
          The denial reasons are provided in the form of claim adjustment reason codes and remittance advice remark codes.

           Args: 
            claim_id (int): the id of the claim you want to lookup
          Returns:
            list[tuple]: A list containing tuples with procedure data. Each tuple  follows this format: (claim_submission_id, cpt_code, modifiers, place_of_service_code, revenue_code, insurance_payout, smartpay_pr, claim_adjustment_reason_codes, remittance_advice_remark_codes)
        """,
    ),
    Tool(
        name="Lookup Approved Claims where CPT Code was approved",
        func=lookup_approved_claims_with_cpt_code,
        description="""
          Useful to find claims where a certain cpt_code was approved. 
          You can use the claim_submission_id to determine which procedures that are in the same claim.
          
          Args: 
            cpt_code (str): the cpt code you want to search for (e.g. 99214)
          Returns:
            list[tuple]: A list containing tuples with procedure data. Each tuple follows this format: (claim_submission_id, list of cpt_codes followed by their modifiers (e.g. 99214-{{95}}), place_of_service_code, revenue_code, insurance_payout, smartpay_pr)
         """,
    ),
]

PREFIX = """ 
  You are an AI assistant that is trying to solve medical billing claims that were denied by insurers. 
  You will be given a claim submission that was denied. 
  First, use the provided tools to understand why the claim was denied. There may be multiple procedures in a claim, so analyze the procedures with 0 insurance_payout and understand their denial reasons.
  Second, based on the denial reason, look for the relevant information in the provided claim submission that may be causing the denial and try to understand what these values mean.
  Third, based on the denial reason, look at patterns in similar claims that were approved by insurers (e.g. the modifiers that were used, etc.).
  Finally, use all this information to explain what is specifically wrong with the claim and how to fix it.
  If there is not enough information to determine the denial reason, then say that you don't have enough information.
"""
llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
denials_copilot = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, agent_kwargs={ 'prefix': PREFIX }, verbose=True)

In [79]:
# Example 1
question = "How should I fix claim 109943?"
denials_copilot.run(question)



> Entering new AgentExecutor chain...
I should start by looking up the denial information for claim 109943 to understand why it was denied.
Action: Lookup Claim Procedure Denial Info
Action Input: 109943
Observation: [
    {
        "claim_submission_id": 109943,
        "cpt_code": "96372",
        "modifiers": [
            "25"
        ],
        "place_of_service_code": "11",
        "revenue_code": null,
        "insurance_payout": 0.0,
        "smartpay_pr": 0.0,
        "claim_adjustment_reason_codes": [
            "4 - The procedure code is inconsistent with the modifier used. Usage: Refer to the 835 Healthcare Policy Identification Segment (loop 2110 Service Payment Information REF), if present."
        ],
        "remittance_advice_remark_codes": [
            " - "
        ]
    }
]
Thought:The denial reason for claim 109943 is "The procedure code is inconsistent with the modifier used." There is a modifier 25 associated with the procedure code 96372. I should now look f

OutputParserException: Could not parse LLM output: `The approved claims with the same procedure code 96372 have different modifiers associated with them. Some claims have no modifiers, while others have modifier 25. I should now analyze the patterns in the approved claims to determine the correct modifier to use for claim 109943.

Action: Analyze patterns in approved claims`

In [62]:
# Example 2
question = "How should I fix claim 275925?"
denials_copilot.run(question)



> Entering new AgentExecutor chain...
I should start by looking up the denial information for claim 275925 to understand why it was denied.
Action: Lookup Claim Procedure Denial Info
Action Input: 275925
Observation: [
    {
        "claim_submission_id": 275925,
        "cpt_code": "99214",
        "modifiers": [
            "25",
            "95"
        ],
        "place_of_service_code": "11",
        "revenue_code": null,
        "insurance_payout": 86.34,
        "smartpay_pr": 0.0,
        "claim_adjustment_reason_codes": null,
        "remittance_advice_remark_codes": null
    },
    {
        "claim_submission_id": 275925,
        "cpt_code": "36415",
        "modifiers": [
            "95"
        ],
        "place_of_service_code": "11",
        "revenue_code": null,
        "insurance_payout": 0.0,
        "smartpay_pr": 0.0,
        "claim_adjustment_reason_codes": [
            "4 - The procedure code is inconsistent with the modifier used. Usage: Refer to the 835 Healt

'Claim 275925 was denied because the procedure code 36415 is inconsistent with the modifier used. To fix the claim, the modifier used for CPT code 36415 should be changed to "{}" (no modifier).'

In [63]:
# Example 3 
question = "How should I fix claim 226518?"
denials_copilot.run(question)



> Entering new AgentExecutor chain...
I should start by looking up the denial information for claim 226518 to understand why it was denied.
Action: Lookup Claim Procedure Denial Info
Action Input: 226518
Observation: [
    {
        "claim_submission_id": 226518,
        "cpt_code": "96372",
        "modifiers": [
            "25"
        ],
        "place_of_service_code": "11",
        "revenue_code": null,
        "insurance_payout": 0.0,
        "smartpay_pr": 0.0,
        "claim_adjustment_reason_codes": [
            "4 - The procedure code is inconsistent with the modifier used. Usage: Refer to the 835 Healthcare Policy Identification Segment (loop 2110 Service Payment Information REF), if present."
        ],
        "remittance_advice_remark_codes": [
            " - "
        ]
    },
    {
        "claim_submission_id": 226518,
        "cpt_code": "J3301",
        "modifiers": [],
        "place_of_service_code": "11",
        "revenue_code": null,
        "insurance_payou

'To fix claim 226518, remove the modifier 25 from procedure code 96372.'

In [64]:
# Example 4 
question = "How should I fix claim 276585?"
denials_copilot.run(question)



> Entering new AgentExecutor chain...
I should start by looking up the denial information for claim 276585 to understand why it was denied.
Action: Lookup Claim Procedure Denial Info
Action Input: 276585
Observation: [
    {
        "claim_submission_id": 276585,
        "cpt_code": "96372",
        "modifiers": [
            "95"
        ],
        "place_of_service_code": "11",
        "revenue_code": null,
        "insurance_payout": 0.0,
        "smartpay_pr": 0.0,
        "claim_adjustment_reason_codes": [
            "4 - The procedure code is inconsistent with the modifier used. Usage: Refer to the 835 Healthcare Policy Identification Segment (loop 2110 Service Payment Information REF), if present."
        ],
        "remittance_advice_remark_codes": [
            "N519 - Invalid combination of HCPCS modifiers."
        ]
    },
    {
        "claim_submission_id": 276585,
        "cpt_code": "99213",
        "modifiers": [
            "25",
            "95"
        ],
      

'Claim 276585 was denied for the following reasons:\n\nProcedure 96372:\n- Denial Reason: The procedure code is inconsistent with the modifier used. Usage: Refer to the 835 Healthcare Policy Identification Segment (loop 2110 Service Payment Information REF), if present.\n- Remittance Advice Remark Code: Invalid combination of HCPCS modifiers.\n\nProcedure J1071:\n- Denial Reason: The procedure code is inconsistent with the modifier used. Usage: Refer to the 835 Healthcare Policy Identification Segment (loop 2110 Service Payment Information REF), if present.\n- Remittance Advice Remark Code: Invalid combination of HCPCS modifiers.\n\nTo fix the claim, the following steps can be taken:\n\n1. For procedure 96372, ensure that the modifier {} is used consistently with the procedure code. Check the 835 Healthcare Policy Identification Segment for any specific requirements related to the modifier usage.\n\n2. For procedure J1071, ensure that the modifier {} is used consistently with the proce

In [65]:
# Example 5 
# NOTE: the claim should add 25 modifier to 90833, and should remove 95 modifier from 99214 (look at POS)
question = "How should I fix claim 518796?"
denials_copilot.run(question)



> Entering new AgentExecutor chain...
I should start by looking up the denial information for claim 518796 to understand why it was denied.
Action: Lookup Claim Procedure Denial Info
Action Input: 518796
Observation: [
    {
        "claim_submission_id": 518796,
        "cpt_code": "90833",
        "modifiers": [],
        "place_of_service_code": "11",
        "revenue_code": null,
        "insurance_payout": 0.0,
        "smartpay_pr": 0.0,
        "claim_adjustment_reason_codes": [
            "4 - The procedure code is inconsistent with the modifier used. Usage: Refer to the 835 Healthcare Policy Identification Segment (loop 2110 Service Payment Information REF), if present."
        ],
        "remittance_advice_remark_codes": [
            " - "
        ]
    },
    {
        "claim_submission_id": 518796,
        "cpt_code": "99214",
        "modifiers": [
            "25",
            "95"
        ],
        "place_of_service_code": "2",
        "revenue_code": null,
       

'To fix claim 518796, the provider should consider adding modifiers 95 and 25 to CPT code 90833 and CPT code 99214. They should also ensure that the place of service code is either 02 or 11.'

In [66]:
# Example 6
# NOTE: this is wrong - LLM is saying use no modifier, but probably need to use 25 modifier. It's only pulling in similar claims that only billed for 90868
question = "How should I fix claim 544924?"
denials_copilot.run(question)



> Entering new AgentExecutor chain...
I should start by looking up the denial information for claim 544924 to understand why it was denied.
Action: Lookup Claim Procedure Denial Info
Action Input: 544924
Observation: [
    {
        "claim_submission_id": 544924,
        "cpt_code": "90868",
        "modifiers": [],
        "place_of_service_code": "11",
        "revenue_code": null,
        "insurance_payout": 0.0,
        "smartpay_pr": 0.0,
        "claim_adjustment_reason_codes": [
            "4 - The procedure code is inconsistent with the modifier used. Usage: Refer to the 835 Healthcare Policy Identification Segment (loop 2110 Service Payment Information REF), if present."
        ],
        "remittance_advice_remark_codes": [
            " - "
        ]
    },
    {
        "claim_submission_id": 544924,
        "cpt_code": "99214",
        "modifiers": [
            "95",
            "25"
        ],
        "place_of_service_code": "2",
        "revenue_code": null,
       

'Claim 544924 was denied for two procedures: CPT code 90868 and CPT code 99214. CPT code 90868 was denied because the procedure code is inconsistent with the modifier used. To fix this, the modifier used for CPT code 90868 should be "{}". CPT code 99214 was denied due to a deductible amount. To fix this, the deductible amount should be paid or resolved with the patient. Additionally, the place of service code for both procedures should be "11" to align with the patterns in approved claims.'

In [68]:
# Example 7
# Note: this is wrong -> LLM is not giving us a specific solution -> no relevant examples. proper soln is to remove the 90 modifier (99214 cannot be sent to an outside lab)
question = "How should I fix claim 194208?"
denials_copilot.run(question)



> Entering new AgentExecutor chain...
I should start by looking up the denial information for claim 194208 to understand why it was denied.
Action: Lookup Claim Procedure Denial Info
Action Input: 194208
Observation: [
    {
        "claim_submission_id": 194208,
        "cpt_code": "99214",
        "modifiers": [
            "25",
            "90"
        ],
        "place_of_service_code": "11",
        "revenue_code": null,
        "insurance_payout": 0.0,
        "smartpay_pr": 0.0,
        "claim_adjustment_reason_codes": [
            "4 - The procedure code is inconsistent with the modifier used. Usage: Refer to the 835 Healthcare Policy Identification Segment (loop 2110 Service Payment Information REF), if present."
        ],
        "remittance_advice_remark_codes": [
            "N644 - Reimbursement has been made according to the bilateral procedure rule."
        ]
    },
    {
        "claim_submission_id": 194208,
        "cpt_code": "36415",
        "modifiers": [],
 

'Claim 194208 was denied for the first procedure with CPT code 99214 and modifiers 25 and 90 because the procedure code is inconsistent with the modifiers used. To fix this, the correct modifiers should be used for the procedure code. Additionally, the denial reason code suggests referring to the 835 Healthcare Policy Identification Segment (loop 2110 Service Payment Information REF) for more information, if present.\n\nThe second procedure with CPT code 36415 was denied due to a deductible amount. To fix this, the deductible amount should be paid or resolved with the insurance company.\n\nOverall, to fix claim 194208, the correct modifiers should be used for the first procedure and the deductible amount for the second procedure should be resolved.'

In [88]:
# Example 7
question = "How should I fix claim 219277?"
denials_copilot.run(question)



> Entering new AgentExecutor chain...
I should start by looking up the denial information for claim 219277 to understand why it was denied.
Action: Lookup Claim Procedure Denial Info
Action Input: 219277
Observation: [
    {
        "claim_submission_id": 219277,
        "cpt_code": "99396",
        "modifiers": [
            "59",
            "25"
        ],
        "place_of_service_code": "11",
        "revenue_code": null,
        "insurance_payout": 0.0,
        "smartpay_pr": 0.0,
        "claim_adjustment_reason_codes": [
            "4 - The procedure code is inconsistent with the modifier used. Usage: Refer to the 835 Healthcare Policy Identification Segment (loop 2110 Service Payment Information REF), if present."
        ],
        "remittance_advice_remark_codes": [
            "N644 - Reimbursement has been made according to the bilateral procedure rule."
        ]
    },
    {
        "claim_submission_id": 219277,
        "cpt_code": "96127",
        "modifiers": [],
 

OutputParserException: Parsing LLM output produced both a final answer and a parse-able action:: Based on the approved claims with the same CPT code (99396), I can see that the insurance payout varies between $98.96 and $246.11. The smartpay_pr also varies between $0.0 and $38.02. 

Thought: Now I can analyze the denial reasons and compare them to the information in the claim submission to identify the specific issues causing the denials.

1. Procedure with CPT code 99396:
   - Denial Reason: The procedure code is inconsistent with the modifier used. Usage: Refer to the 835 Healthcare Policy Identification Segment (loop 2110 Service Payment Information REF), if present.
   - Remittance Advice Remark Code: Reimbursement has been made according to the bilateral procedure rule.

   Thought: The denial reason suggests that the procedure code is inconsistent with the modifier used. I should check the modifiers in the claim submission for this procedure.

   Action: Lookup Claim Procedure De